# Trabajo Practico Nº1 - Parte 3: Comunicación y Sincronismo

## Información General

* **Universidad Nacional de la Matanza**
* Materia: Programación Concurrente
* Cuatrimestre: Segundo Cuatrimestre, Año 2024
* Profesores:
    * ADAGIO, MATIAS EZEQUIEL
    * CARNUCCIO, ESTEBAN
    * HIRSCHFELDT, DARIO
    * PALOMO, MAXIMO FACUNDO
    * VOLKER MARIANO LEONARDO
* Grupo: M4


* Integrantes:
    * ANTONIOLI, IVÁN OSCAR
    * DI NICCO, LUIS DEMETRIO
    * SANDOVAL VASQUEZ, JUAN LEANDRO
    * TIGANI MARTIN SEBASTIAN
    * VILLCA, LUIS ALBERTO

## Consigna

**Fecha de entrega**: 25/09/2024

**Forma de entrega**:
Se debe generar un informe que contenga los siguientes puntos:

*   **Carátula**: Con los integrantes del grupo.
*   **Link a un repositorio de GitHub**: En donde se encuentre el archivo del cuaderno de Colab generado para este Trabajo Práctico. Este archivo debe ser con la extension ipynb. También se debe subir a GitHub el código fuente.
*   **Conclusiones**: En esta sección se debe describir las dificultadas que encontraron al realizar el trabajo práctico.

Entregar el informe por plataforma de MIeL. Este debe ser en formato .pdf, con nombre TP1_Parte3_NumeroDelGrupo.pdf.

**Enunciado**: En un supermercado existen **dos (2) repositores** que reponen los productos de una góndola con capacidad máxima de **diez (10) productos**, dichos repositores trabajan con alternancia estricta, es decir que una vez repone uno, luego el otro y así sucesivamente. También existen **N clientes**: que van tomando los productos de las góndolas (1 a 2 productos de manera aleatoria). Cuando la góndola se vacía, el repositor repone la totalidad de productos. Cabe destacar que mientras se están reponiendo los productos, los clientes no pueden tomarlos, así como también si un cliente no obtuvo la cantidad de productos que necesitaba, esperará hasta que la góndola se complete.

Desarrolle un programa utilizando algún método de comunicación y/o sincronización que represente el funcionamiento anteriormente planteado para un número N de clientes donde N será pasado como parámetro al programa.  

## Criterios a tener en cuenta

* [Coding Standard](https://google.github.io/styleguide/)
* Ajustes específicos (Ej. llaves en la misma columna)
* Espacios en lugar de tabulaciones (2 espacios)
* Métodos/funciones con menos de 15 líneas.
* Patrones de diseño
* No debe haber números mágicos, uso de constantes descriptivas.

## Resolución

### Código

Clase Cliente

In [ ]:
%%writefile Cliente.java
public class Cliente extends Thread
{

  private int nombre;
  private int cantidadAComprar;

  public Cliente(int nombre, int cantidadAComprar)
  {
    this.nombre = nombre;
    this.cantidadAComprar = cantidadAComprar;
  }

  public void run()
  {
    try
    {
      while (true)
      {
        Gondola.accesoProductos.acquire();
        if (Gondola.productos >= cantidadAComprar)
        {
          Gondola.productos -= cantidadAComprar;
          System.out.println("-----------------------------------------------------------");
          System.out.println("Cliente: " + nombre + " esta comprando " + cantidadAComprar + " productos.");
          System.out.println("Productos en gondola despues de la compra: " + Gondola.productos);
          System.out.println("-----------------------------------------------------------");
          Gondola.accesoProductos.release();
          break;
        }
        else
        {
          Gondola.accesoProductos.release();
          sleep(1000);
        }
      }

    }
    catch (InterruptedException e)
    {
      e.printStackTrace();
    }
  }
}

Writing Cliente.java


Clase constantes

In [ ]:
%%writefile Constantes.java
public class Constantes
{
  public static final int CANTIDAD_MAXIMA_PRODUCTOS = 10;
  public static final int PRIMER_TURNO = 1;
  public static final int SEGUNDO_TURNO = 2;
}

Overwriting Constantes.java


Clase Repositor

In [ ]:
%%writefile Repositor.java
public class Repositor extends Thread
{

  private static int turno = Constantes.PRIMER_TURNO;

  private int numero;
  private int cantidadAReponer;

  public Repositor(int numero, int cantidadAReponer)
  {
    this.numero = numero;
    this.cantidadAReponer = cantidadAReponer;
  }

  public void cambiarTurno()
  {
    turno = Repositor.turno == Constantes.PRIMER_TURNO ? Constantes.SEGUNDO_TURNO : Constantes.PRIMER_TURNO;
  }

  public void run()
  {
    try
    {
      while (Gondola.reposicionActiva)
      {
        Gondola.accesoProductos.acquire();
        if (Gondola.productos < Constantes.CANTIDAD_MAXIMA_PRODUCTOS && turno == numero)
        {
          int cantidadReponer = Math.min(Constantes.CANTIDAD_MAXIMA_PRODUCTOS - Gondola.productos, cantidadAReponer);
          Gondola.productos += cantidadReponer;
          System.out.println("-----------------------------------------------------------");
          System.out.println("Repositor: " + numero + " esta reponiendo " + cantidadReponer + " productos.");
          System.out.println("Productos en gondola despues de la reposición: " + Gondola.productos);
          System.out.println("-----------------------------------------------------------");
          cambiarTurno();
          Gondola.accesoProductos.release();
        }
        else
        {
          Gondola.accesoProductos.release();
          sleep(1000);
        }
      }

    }
    catch (InterruptedException e)
    {
      e.printStackTrace();
    }
  }
}

Writing Repositor.java


Clase Gondola

In [ ]:
%%writefile Gondola.java
import java.util.concurrent.Semaphore;

public class Gondola
{

  public static int productos = 0;
  static Semaphore accesoProductos = new Semaphore(1);
  static boolean reposicionActiva = true;

  public Cliente[] obtenerClientes(int cantidadClientes)
  {
    Cliente[] clientes = new Cliente[cantidadClientes];
    for (int i = 0; i < cantidadClientes; i++)
    {
      clientes[i] = new Cliente(i, (int) (Math.random() * 2) + 1);
      clientes[i].start();
    }
    return clientes;
  }
}

Writing Gondola.java


Clase Main

In [ ]:
%%writefile Main.java
public class Main
{
  public static void main(String[] args) throws InterruptedException
  {

    if (args.length < 1 || Integer.parseInt(args[0]) < 0)
    {
      System.out.println("La cantidad de clientes debe ser positiva");
      return;
    }

    int numClientes = Integer.parseInt(args[0]);

    Repositor repo1 = new Repositor(1, Constantes.CANTIDAD_MAXIMA_PRODUCTOS);
    Repositor repo2 = new Repositor(2, Constantes.CANTIDAD_MAXIMA_PRODUCTOS);

    repo1.start();
    repo2.start();

    Gondola gondola = new Gondola();

    Cliente[] clientes = gondola.obtenerClientes(numClientes);

    for (int i = 0; i < numClientes; i++)
    {
      clientes[i].join();
    }

    Gondola.reposicionActiva = false;

    repo1.join();
    repo2.join();
  }
}


Writing Main.java


### Compilación

In [ ]:
!javac Main.java Gondola.java Repositor.java Constantes.java Cliente.java

### Ejecución

In [ ]:
!java Main 10

-----------------------------------------------------------
Repositor: 1 esta reponiendo 10 productos.
Productos en gondola despues de la reposición: 10
-----------------------------------------------------------
-----------------------------------------------------------
Cliente: 4 esta comprando 1 productos.
Productos en gondola despues de la compra: 9
-----------------------------------------------------------
-----------------------------------------------------------
Cliente: 6 esta comprando 1 productos.
Productos en gondola despues de la compra: 8
-----------------------------------------------------------
-----------------------------------------------------------
Cliente: 5 esta comprando 2 productos.
Productos en gondola despues de la compra: 6
-----------------------------------------------------------
-----------------------------------------------------------
Cliente: 7 esta comprando 2 productos.
Productos en gondola despues de la compra: 4
-------------------------------